# Extraction of pixel values from ROI using sum projection of z-stack image

This notebook computes sum projections of z-stack images. The user can then draw ROIs, as well as background areas, manually using the bebi103.image.record_clicks function developed by Justin Bois. The notebook then normalizes each ROI pixel value against the mean pixel value in the background (BG) area. It then saves the raw integrated density (sum of all BG normalized pixel values inside the ROI), as well as the integrated density (product of ROI area (number of pixels in ROI) and mean BG normalized pixel value inside the ROI). Image matrices, ROI coordinates, and pixel values are saved to a dictionary ("dicts_sum") and continuously linked to their sample#, channel, condition, and original filename.

#### Do not click "Run all", but stop at "STOP" cells to draw the ROIs BEFORE continuing to run the notebook.

#### Name tif files as follows: 
X_samplecondition_sample_channel , where "X" is optional and can be filled as desired by the user (do not use spaces).  
- "Samplecondition" can be a string of text separated by "-", e.g. "GCaMP-30C-6d_sample_channel" (Do not use spaces or "_" within the condition, else part of it will be cut out in the dictionary.)  
- "Sample" and "channel" can be a simple number or a description, e.g.: "samplecondition_1_1" or "samplecondition_Brain1_DAPI". (Do not use spaces or "_" within the sample or channel.)  

Written by Laura Luebbert, 20th of August 2020.  

Modified on: / 

## Define the parameters:

### Define the directory containing the tif files:

In [ ]:
data_dir = "/Users/"

### General saving options:

In [ ]:
# Define the directory the pickled dictionaries will be saved to (inside folder named after condition folder)
saving_dir = data_dir

### Sum projection images saving options:

In [ ]:
# Do you want to save the sum projection of the images? (Define as "yes" or "no".)
save_sum_ims = "yes"

#### If yes, define the following (if not you can skip the following parameters and begin running the notebook):

Saving options:

In [ ]:
# Directory the sum projections of the images will be saved to (inside a new folder named after the condition folder)
saving_dir_sum = data_dir

# Set the file format the images should be saved as
file_format = "tif"

Scale bar options:

In [ ]:
# Turn scale bar on/off (define as "yes" or "no")
scale_bar = "yes"

# Set scalebar color ("white" or "black")
scale_bar_color = "white"

# Define the desired scale bar size and width in length unit
scale_bar_length = 100
scale_bar_width = 5

# Define position of scale bar:
# Distance from top (Distance in % of total length of image)
y_pos = 95
# Distance of rightmost end of scale bar from left end of image (Distance in % of total length of image)
x_pos = 95

Define microscope and magnification (to get interpixel distance) OR define interpixel distance directly if different microscope/magnification used:

In [ ]:
# Define microscope used ("Lois", "LSM800" or "other")
microscope = "Lois"

# Define magnification ("20" or "other")
magnification = "20"

# Uncomment (remove the #) and define interpixel distance if different microscope/magnification used:
# interpixel_distance = 

length_units = "µm"

___

___

In [ ]:
# %load_ext blackcellmagic
%config InlineBackend.figure_format = 'retina'

In [ ]:
# Tools to read in the image files and filenames
import glob
import os
import re 

# Calculation and data frame tools
import numpy as np
import pandas as pd

# Image processing tools
import bebi103
import skimage
import skimage.io
import skimage.morphology

# Tools to create new folders
from pathlib import Path

# Tools to save dictionaries
import pickle

# (Interactive) plotting tools
import bokeh
import bokeh_catplot
bokeh.io.output_notebook()

___

# Load in the data

In [ ]:
# Glob string for images (loads all .tif files)
im_glob = os.path.join(data_dir, "*.tif")

# Get list of files in directory
im_list = sorted(glob.glob(im_glob))

# Display first 10 entries in list of filenames
im_list[:10]

Create a nested dictionary with information about each sample coupled to the z-stack image matrix, as well as space for matrices added later:

In [ ]:
dicts_sum = {}

for i in range(len(im_list)):
    # Get sample condition
    condition = im_list[i].split("/")[-1].split("_")[-3]
    
    # Make sure we do not overwrite a previous dictionary entry    
    if dicts_sum.get(condition) == None:
        dicts_sum[condition] = {}
    
    # Get sample number
    sample = im_list[i].split("/")[-1].split("_")[-2]
    
    # Make sure we do not overwrite a previous dictionary entry
    if dicts_sum.get(condition, {}).get(sample) == None:
        dicts_sum[condition][sample] = {}
    
    # Get channel number    
    channel = im_list[i].split("/")[-1].split("_")[-1].split(".")[0]
    
    # Add empty arrays to dictionary to populate with images later
    dicts_sum[condition][sample][channel] = {
        "matrix_orig": [],  # Original image z-stack matrix 
        "matrix_sum": [],   # Image matrix of sum projection image 
        "matrix_8": [],     # Image matrix of sum projection image as 8-bit 
        "matrix_8_SB": [],  # Image matrix of sum projection image as 8-bit with scale bar 
        "filename": [],     # The original filename, just in case
        "clicks": [],       # The coordinates of "clicks" defining the ROI 
        "rois": [],         # ROI clicks converted to an ROI (polygon) 
        "clicks_bg": [],    # The coordinates of "clicks" defining the background area to normalize against 
        "rois_bg": [],      # BG area clicks converted to a polygon
        "mean_int": [],     # Mean pixel value in ROI
        "mean_int_bg": [],  # Mean pixel value in BG area
        "raw_int_den": [],  # Raw integrated density of ROI (sum of all BG normalized pixel values inside the ROI) 
        "int_den": []       # Integrated density of ROI (product of ROI area and mean BG normalized pixel value inside the ROI)
    }
    
    # Populate dictionary with original image matrix
    dicts_sum[condition][sample][channel]["matrix_orig"] = skimage.io.imread(im_list[i])
    
    # Populate dictionary with original filename
    dicts_sum[condition][sample][channel]["filename"] = im_list[i].split("/")[-1].split(".")[-2]

___

# Get the interpixel distance

In [ ]:
if microscope == "LSM800":
    if magnification == "20":
        interpixel_distance = 0.3119629
        
if microscope == "Lois":
    if magnification == "20":
        interpixel_distance = 0.690534

___

# Create sum projections for fluorescence analysis

Compute sum of frames for each image:

In [ ]:
for k1_condition, v1_sample in dicts_sum.items():
    for k2_sample, v2_channel in v1_sample.items():
        for k3_channel, im in v2_channel.items():
            # Convert dict entry to array
            image = np.array(im["matrix_orig"])

            # Sum z-project using numpy by definining the axis over which to sum the elements.
            summed_image = image.sum(axis=0)

            # Linearly scale down to 16-bit.
            summed_image = (summed_image/summed_image.max())*65535

            # Save summed image to dictionary
            dicts_sum[k1_condition][k2_sample][k3_channel]["matrix_sum"] = summed_image

Display one sum projection image using skimage:

In [ ]:
skimage.io.imshow(dicts_sum[k1_condition][k2_sample][k3_channel]["matrix_sum"])

___

# Save sum projection images 
(Turn this option on at the top of this notebook under "parameters".)

#### Scale down images:

To save the images using skimage, we need to scale them down to 8 bit.

In [ ]:
if save_sum_ims == "yes":
    # Scale down images to 8 bits
    for k1_condition, v1_sample in dicts_sum.items():
        for k2_sample, v2_channel in v1_sample.items():
            for k3_channel, im in v2_channel.items():
                # Linearly scale image down to 8-bit.
                image = (im["matrix_sum"] / im["matrix_sum"].max()) * 255

                # Change list to array and change type to 8-bit array.
                image = np.array(image)
                image = image.astype(np.uint8)

                dicts_sum[k1_condition][k2_sample][k3_channel]["matrix_8"] = image

#### Burn in scale bars:

Scale bar is burned into image by changing the pixel value to 1000 (white scale bar) or 0 (black scale bar) in scale bar area defined in the corresponding "parameters" cell above:

In [ ]:
if save_sum_ims == "yes":
    if scale_bar == "yes":

        scalebar = 1 / interpixel_distance * scale_bar_length
        scalebar_width = 1 / interpixel_distance * scale_bar_width

        if scale_bar_color == "white":
            for k1_condition, v1_sample in dicts_sum.items():
                for k2_sample, v2_channel in v1_sample.items():
                    for k3_channel, im in v2_channel.items():
                        y_value = int((im["matrix_8"].shape[0]/100)*y_pos)
                        x_value = int((im["matrix_8"].shape[1]/100)*x_pos) 

                        im["matrix_8"][y_value : y_value + int(scalebar_width), x_value - int(scalebar) : x_value] = 255

                        # Populate dict with 8-bit images containing a scale bar
                        dicts_sum[k1_condition][k2_sample][k3_channel]["matrix_8_SB"] = im["matrix_8"]

        elif scale_bar_color == "black":
            for k1_condition, v1_sample in dicts_sum.items():
                for k2_sample, v2_channel in v1_sample.items():
                    for k3_channel, im in v2_channel.items():
                        y_pos = int((im["matrix_8"].shape[0]/100)*y_pos)
                        x_pos = int((im["matrix_8"].shape[1]/100)*x_pos)

                        im["matrix_8"][y_pos : y_pos + int(scalebar_width), x_pos - int(scalebar) : x_pos] = 0

                        # Populate dict with 8-bit images containing a scale bar
                        dicts_sum[k1_condition][k2_sample][k3_channel]["matrix_8_SB"] = im["matrix_8"]

    else:
        for k1_condition, v1_sample in dicts_sum.items():
            for k2_sample, v2_channel in v1_sample.items():
                for k3_channel, im in v2_channel.items():

                    dicts_sum[k1_condition][k2_sample][k3_channel]["matrix_8_SB"] = im["matrix_8"]

Display one image with scale bar:

In [ ]:
if save_sum_ims == "yes":
    skimage.io.imshow(dicts_sum[k1_condition][k2_sample][k3_channel]["matrix_8_SB"])

### Create sum projection folder:

Create folder in saving directory to save maximum projections to:

In [ ]:
if save_sum_ims == "yes":
    path = ("{}/{}_sum_projections").format(saving_dir, im_list[0].split("/")[-2])
    Path(path).mkdir(parents=True, exist_ok=True)

### Save all images with a scale bar:

In [ ]:
if save_sum_ims == "yes":
    for k1_condition, v1_sample in dicts_sum.items():
        for k2_sample, v2_channel in v1_sample.items():
            for k3_channel, im in v2_channel.items():
                skimage.io.imsave(
                    ("{}/{}_sum.{}").format(path, im["filename"], file_format),
                    im["matrix_8_SB"],
                    plugin=None,
                    check_contrast=True,
                )

___

# Define ROIs 

### Click points around area of interest to define an ROI (only draw one ROI)
#### If you get an error using the bebi103.image.record_clicks package below, make sure to use websocket 8888. (Define localhost:8888/ in your URL.)

"By default in bebi103.image.imshow(), the flip kwarg is set to True. This means that the image is flipped vertically so that it appears rightside up, since the indexing convention for images (2D arrays) starts in the upper left corner, but for axes, the origin is conventionally in the lower left corner. For recording clicks for use in image processing, you should be sure that the flip kwarg is False." - JB

In [ ]:
for k1_condition, v1_sample in dicts_sum.items():
    for k2_sample, v2_channel in v1_sample.items():
        for k3_channel, im in v2_channel.items():
            temp = bebi103.image.record_clicks(
                dicts_sum[k1_condition][k2_sample][k3_channel]["matrix_sum"],
                frame_height=500,
                title=dicts_sum[k1_condition][k2_sample][k3_channel]["filename"],
                flip=False
            )

            # Save clicks to dictionary
            dicts_sum[k1_condition][k2_sample][k3_channel]["clicks"] = temp

# STOP

Convert clicks to a tidy data frame:

In [ ]:
for k1_condition, v1_sample in dicts_sum.items():
    for k2_sample, v2_channel in v1_sample.items():
        for k3_channel, im in v2_channel.items():
            temp = dicts_sum[k1_condition][k2_sample][k3_channel]["clicks"].to_df()

            # Add "roi" column (in this case there is just one ROI per sample with number "0")
            temp["roi"] = 0

            # Save clicks to dictionary as tidy data frame (this will overwrite the previously saved version of the clicks!)
            dicts_sum[k1_condition][k2_sample][k3_channel]["clicks"] = temp

Use the bebi103.image.verts_to_roi function to convert the set of vertices (clicks) that define a polygon to a region of interest (the inside of the polygon):

In [ ]:
for k1_condition, v1_sample in dicts_sum.items():
    for k2_sample, v2_channel in v1_sample.items():
        for k3_channel, im in v2_channel.items():
            dicts_sum[k1_condition][k2_sample][k3_channel]["rois"] = [bebi103.image.verts_to_roi(dicts_sum[k1_condition][k2_sample][k3_channel]["clicks"][['x', 'y']].values, 
                                        dicts_sum[k1_condition][k2_sample][k3_channel]["matrix_sum"].shape[0], 
                                        dicts_sum[k1_condition][k2_sample][k3_channel]["matrix_sum"].shape[1])
            for _, g in dicts_sum[k1_condition][k2_sample][k3_channel]["clicks"].groupby('roi')]

Check that the ROIs are correct:

In [ ]:
plots = []

for k1_condition, v1_sample in dicts_sum.items():
    for k2_sample, v2_channel in v1_sample.items():
        for k3_channel, im in v2_channel.items():
            # The function above return 3 representation of our ROI:
            # roi = Boolean matrix (mask) in the size of my original image with "True" values where the ROI is
            # roi_bbox = Bounding box around ROI
            # roi_box = Boolean matrix in the size of the bounding box (roi_bbox) with "True" values where the ROI is
            roi, roi_bbox, roi_box = dicts_sum[k1_condition][k2_sample][k3_channel]["rois"][0]

            # Make grayscale image that is now RBG/CMY
            im = np.dstack(3 * [skimage.img_as_float(dicts_sum[k1_condition][k2_sample][k3_channel]["matrix_sum"])])

            # Max out first channel
            im[roi, 0] = im.max()
            plots.append(
                bebi103.image.imshow(
                    im,
                    title=dicts_sum[k1_condition][k2_sample][k3_channel]["filename"],
                    frame_height=250,
                    cmap="rgb",
                    flip=False,
                )
            )

# Look at the images
bokeh.io.show(bokeh.layouts.gridplot(plots, ncols=3))

### Save mean fluorescence value from ROI

In [ ]:
for k1_condition, v1_sample in dicts_sum.items():
    for k2_sample, v2_channel in v1_sample.items():
        for k3_channel, im in v2_channel.items():
            roi, roi_bbox, roi_box = dicts_sum[k1_condition][k2_sample][k3_channel]["rois"][0]
            im = dicts_sum[k1_condition][k2_sample][k3_channel]["matrix_sum"][roi]

            # Save fluorescence intensity of ROI to dictionary
            dicts_sum[k1_condition][k2_sample][k3_channel]["mean_int"] = im.mean()

___

# Define background fluorescence area

Define a small square outside of the ROI which is representative of the background fluorescence. Each pixel of the ROI will be divided by the average pixel value of the background before the overall fluorecence in the ROI is summed.  

In [ ]:
for k1_condition, v1_sample in dicts_sum.items():
    for k2_sample, v2_channel in v1_sample.items():
        for k3_channel, im in v2_channel.items():
            temp = bebi103.image.record_clicks(
                dicts_sum[k1_condition][k2_sample][k3_channel]["matrix_sum"],
                frame_height=500,
                title=dicts_sum[k1_condition][k2_sample][k3_channel]["filename"],
                flip=False
            )

            # Save clicks to dictionary
            dicts_sum[k1_condition][k2_sample][k3_channel]["clicks_bg"] = temp

# STOP

Convert clicks to a tidy data frame:

In [ ]:
for k1_condition, v1_sample in dicts_sum.items():
    for k2_sample, v2_channel in v1_sample.items():
        for k3_channel, im in v2_channel.items():
            temp = dicts_sum[k1_condition][k2_sample][k3_channel]["clicks_bg"].to_df()

            # Add "roi" column (in this case there is just one ROI per sample with number "0")
            temp["roi"] = 0

            # Save clicks to dictionary as tidy data frame (this will overwrite the previously saved version of the clicks!)
            dicts_sum[k1_condition][k2_sample][k3_channel]["clicks_bg"] = temp

Use the bebi103.image.verts_to_roi function to convert the set of vertices (clicks) that define a polygon to a region of interest (the inside of the polygon):

In [ ]:
for k1_condition, v1_sample in dicts_sum.items():
    for k2_sample, v2_channel in v1_sample.items():
        for k3_channel, im in v2_channel.items():
            dicts_sum[k1_condition][k2_sample][k3_channel]["rois_bg"] = [bebi103.image.verts_to_roi(dicts_sum[k1_condition][k2_sample][k3_channel]["clicks_bg"][['x', 'y']].values, 
                                        dicts_sum[k1_condition][k2_sample][k3_channel]["matrix_sum"].shape[0], 
                                        dicts_sum[k1_condition][k2_sample][k3_channel]["matrix_sum"].shape[1])
            for _, g in dicts_sum[k1_condition][k2_sample][k3_channel]["clicks_bg"].groupby('roi')]

Check that background areas were drawn correctly:

In [ ]:
plots = []

for k1_condition, v1_sample in dicts_sum.items():
    for k2_sample, v2_channel in v1_sample.items():
        for k3_channel, im in v2_channel.items():
            # The function above return 3 representation of our ROI:
            # roi = Boolean matrix (mask) in the size of my original image with "True" values where the ROI is
            # roi_bbox = Bounding box around ROI
            # roi_box = Boolean matrix in the size of the bounding box (roi_bbox) with "True" values where the ROI is
            roi, roi_bbox, roi_box = dicts_sum[k1_condition][k2_sample][k3_channel]["rois_bg"][0]

            # Make grayscale image that is now RBG/CMY
            im = np.dstack(3 * [skimage.img_as_float(dicts_sum[k1_condition][k2_sample][k3_channel]["matrix_sum"])])

            # Max out pixel value in first channel
            im[roi, 0] = im.max()

            plots.append(
                bebi103.image.imshow(
                    im,
                    title=dicts_sum[k1_condition][k2_sample][k3_channel]["filename"],
                    frame_height=250,
                    cmap="rgb",
                    flip=False,
                )
            )

# Look at the images
bokeh.io.show(bokeh.layouts.gridplot(plots, ncols=3))

# Save mean fluorescence value from background area

In [ ]:
for k1_condition, v1_sample in dicts_sum.items():
    for k2_sample, v2_channel in v1_sample.items():
        for k3_channel, im in v2_channel.items():
            roi, roi_bbox, roi_box = dicts_sum[k1_condition][k2_sample][k3_channel]["rois_bg"][0]
            im = dicts_sum[k1_condition][k2_sample][k3_channel]["matrix_sum"][roi]

            # Save fluorescence intensity of ROI to dictionary
            dicts_sum[k1_condition][k2_sample][k3_channel]["mean_int_bg"] = im.mean()

___

# Compute raw integrated density and integrated density of ROI from BG normalized pixel values

In [ ]:
for k1_condition, v1_sample in dicts_sum.items():
    for k2_sample, v2_channel in v1_sample.items():
        for k3_channel, im in v2_channel.items():    
            roi, roi_bbox, roi_box = dicts_sum[k1_condition][k2_sample][k3_channel]["rois"][0]
            im = dicts_sum[k1_condition][k2_sample][k3_channel]["matrix_sum"][roi]

            pixel_norm = []

            # Divide each pixel value by the average background pixel value
            for pixel in im:
                pixel_norm.append(pixel / dicts_sum[k1_condition][k2_sample][k3_channel]["mean_int_bg"])

            # Save raw integrated density of ROI to dictionary
            dicts_sum[k1_condition][k2_sample][k3_channel]["raw_int_den"] = sum(pixel_norm)
            
            ## Save integrated density of ROI to dictionary
            # Calculate the number of pixels in the ROI
            roi_area = 0
            for i in range(len(roi)):
                x = np.sum(roi[i])
                roi_area = roi_area + x
                
            pixel_norm = np.array(pixel_norm)
                
            dicts_sum[k1_condition][k2_sample][k3_channel]["int_den"] = pixel_norm.mean() * roi_area

___

# Pickle sum fluorescence dictionary for later use
This dictionary contains the sum fluorescence images, names of the images, clicks as data frame, rois and the sum of the pixel intensity values inside the roi.   

In [ ]:
path = ("{}").format(saving_dir)

In [ ]:
# Use pickle to save dictionaries

# The advantage of HIGHEST_PROTOCOL is that files get smaller. This makes unpickling sometimes much faster.
with open(
    ("{}/{}_fluorescence_analysis.pickle").format(path, im_list[0].split("/")[-2]),
    "wb",
) as handle:
    pickle.dump(dicts_sum, handle, protocol=pickle.HIGHEST_PROTOCOL)

___

# Create a data frame and csv file for comparison of pixel values across conditions

In [ ]:
df = pd.DataFrame()

for k1_condition, v1_sample in dicts_sum.items():
    for k2_sample, v2_channel in v1_sample.items():
        for k3_channel, im in v2_channel.items(): 
            df = df.append(
                {"Filename" : dicts_sum[k1_condition][k2_sample][k3_channel]["filename"],
                 "Condition" : k1_condition,
                 "Sample" : k2_sample,
                 "Channel" : k3_channel,
                 "Condition + Channel" : (k1_condition + " (" + k3_channel + ")"),
                 "Mean pixel value in ROI" : dicts_sum[k1_condition][k2_sample][k3_channel]["mean_int"],
                 "Mean pixel value in BG area" : dicts_sum[k1_condition][k2_sample][k3_channel]["mean_int_bg"],
                 "Raw integrated density" : dicts_sum[k1_condition][k2_sample][k3_channel]["raw_int_den"],
                 "Integrated density" : dicts_sum[k1_condition][k2_sample][k3_channel]["int_den"]}, ignore_index=True)           

In [ ]:
# Display top of data frame
df.head()

___

# Rescale fluorescence values from 0 to 1

Since the intensity units are arbitrary, we can subtract the mean and rescale the data so they go from 0 to 1:

In [ ]:
fluorescence_norm = []

for i, value in enumerate(df["Integrated density"]):
    fluorescence_norm.append(
        (value - df["Integrated density"].min())
        / (df["Integrated density"].max() - df["Integrated density"].min())
    )

df["Integrated density 0-1"] = fluorescence_norm

In [ ]:
# Sort the data frame by condition
df = pd.DataFrame.sort_values(df, "Condition", ascending=False)

In [ ]:
# Display top of data frame
df.head()

___

# Save dataframe to csv

In [ ]:
df.to_csv("{}/{}_fluorescence_analysis.csv".format(saving_dir, data_dir.split("/")[-1]), index=False)

___

# Plotting

In [ ]:
# Sort the data frame by channel to separate them in plots
df = pd.DataFrame.sort_values(df, "Channel", ascending=False)

Example scatter plot using Matplotlib:

In [ ]:
names = df["Condition + Channel"]
values = df["Integrated density 0-1"]

color = "black"
dotsize = 20

fig, ax = plt.subplots(figsize=(20, 7))

plt.scatter(names, values, c=color, s=dotsize)

# Define figure title
ax.set_title('Relative fluorescence units across conditions', weight='bold', size=17)

# Set axis labels
fontsize = 13
fontweight = 'bold'
fontproperties = {'weight':fontweight, 'size':fontsize}
ax.set_xlabel('Condition', fontproperties)
ax.set_ylabel('Relative Fluorescence Units', fontproperties)

Example boxplot using Matplotlib:

In [ ]:
names = df["Condition"]
values = df["Integrated density 0-1"]

color = "black"
dotsize = 20

fig, ax = plt.subplots(figsize=(20, 7))

labels = names.unique()

# Create boxplot
boxplot = []
for i in names.unique():
    boxplot.append(df.loc[df['Condition']==i, :]["Integrated density 0-1"].values)

# ax.boxplot(boxplot) 
b_plot = ax.boxplot(boxplot, labels=labels) 
for patch in b_plot["boxes"]:
    patch.set_alpha(0.2)

# Overlay scatter plot
ax.scatter(pd.factorize(names)[0]+1, values, s=dotsize, zorder=10, c=color)

# Define figure title
ax.set_title('Relative fluorescence units across conditions',weight='bold',size=17)

# Set axis labels
fontsize = 13
fontweight = 'bold'
fontproperties = {'weight':fontweight, 'size':fontsize}
ax.set_xlabel('Condition', fontproperties)
ax.set_ylabel('Relative Fluorescence Units', fontproperties)

Example barplot using Matplotlib:

In [ ]:
names = df["Condition + Channel"]
values = df["Integrated density 0-1"]

fig, ax = plt.subplots(figsize=(20, 7))

plt.bar(names, values)

# Define figure title
ax.set_title('Relative fluorescence units across conditions', weight='bold', size=17)

# Set axis labels
fontsize = 13
fontweight = 'bold'
fontproperties = {'weight':fontweight, 'size':fontsize}
ax.set_xlabel('Condition', fontproperties)
ax.set_ylabel('Relative Fluorescence Units', fontproperties)

___

# Computing environment

In [ ]:
%load_ext watermark

%watermark -v -p re,numpy,pandas,bebi103,skimage,matplotlib,panel,bokeh,bokeh_catplot,jupyterlab